# 05 Event Generation

This notebook contains examples on how to generate cascades, tracks and starting tracks with the Olympus software package.

First, we got to import some packages.

In [ ]:
import os

from ananke.configurations.collection import HDF5StorageConfiguration
from ananke.visualisation.event import draw_hit_histogram, draw_hit_distribution
from ananke.visualisation.detector import get_detector_scatter3ds
from olympus.configuration.generators import EventGeneratorConfiguration
from olympus.configuration.generators import GenerationConfiguration
from olympus.event_generation.medium import MediumEstimationVariant
from olympus.configuration.generators import UniformSpectrumConfiguration
from ananke.models.collection import Collection
from ananke.schemas.event import EventType
from olympus.configuration.photon_propagation import MockPhotonPropagatorConfiguration

from olympus.configuration.generators import DatasetConfiguration

from ananke.configurations.presets.detector import single_line_configuration
from olympus.event_generation.generators import generate

## Creating the configuration

To create events, we need a detector, a photon propagation, and a storage configuration.

In [ ]:
detector_configuration = single_line_configuration
data_path = 'data/example/05_event_generation.h5'

storage_configuration = HDF5StorageConfiguration(
        data_path=data_path,
        read_only=False
)

# This is optional
photon_propagator_configuration = MockPhotonPropagatorConfiguration(
    resolution=18000,
    medium=MediumEstimationVariant.PONE_OPTIMISTIC,
    max_memory_usage=int(2147483648 / 4) # Great to overcome memory issues
)

Next up, we define our generation configurations for all types.

In [ ]:
cascade_generation_configuration = GenerationConfiguration(
    generator=EventGeneratorConfiguration(
        type=EventType.CASCADE,
        spectrum=UniformSpectrumConfiguration(
            log_minimal_energy=2.0,
            log_maximal_energy=5.5
        ),
        source_propagator=photon_propagator_configuration
    ),
    number_of_samples=3
)

track_generation_configuration = GenerationConfiguration(
    generator=EventGeneratorConfiguration(
        type=EventType.REALISTIC_TRACK,
        spectrum=UniformSpectrumConfiguration(
            log_minimal_energy=2.0,
            log_maximal_energy=5.5
        ),
        source_propagator=photon_propagator_configuration
    ),
    append=True, # Important as otherwise no extra records are generated
    number_of_samples=3
)

starting_track_generation_configuration = GenerationConfiguration(
    generator=EventGeneratorConfiguration(
        type=EventType.STARTING_TRACK,
        spectrum=UniformSpectrumConfiguration(
            log_minimal_energy=2.0,
            log_maximal_energy=5.5
        ),
        source_propagator=photon_propagator_configuration
    ),
    append=True,
    number_of_samples=3
)

Now, lets put it all together

In [ ]:
configuration = DatasetConfiguration(
    detector=detector_configuration,
    generators=[
        cascade_generation_configuration,
        track_generation_configuration,
        starting_track_generation_configuration,
    ],
    storage=storage_configuration
)

## Creating the collection

Once, you have the complete configuration, creating the events is simple.

In [ ]:
# Optional, but to keep the data lean
try:
    os.remove(data_path)
except OSError:
    pass

collection = generate(configuration)


In [ ]:
if 'collection' not in globals():
    collection = Collection(storage_configuration)

## Exploring the events

As we have our events, let's have a look at them. First, let's look at the statistics

In [ ]:
with collection:
    statistics = collection.get_record_statistics()

statistics.df.index

### Saving the Event images

In [ ]:
with collection:
    records = collection.storage.get_records()
    hits = collection.storage.get_hits()
    sources = collection.storage.get_sources()
    detector = collection.storage.get_detector()
    
records.df

In [ ]:
image_path = 'data/event_generation_images/'
isExist = os.path.exists(image_path)

if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(image_path)


for record_id in records.record_ids:
    record = records.get_by_record_ids(record_id)
    record_hits = hits.get_by_record_ids(record_id)
    record_sources = sources.get_by_record_ids(record_id)
    fig = draw_hit_distribution(record_hits)
    fig.savefig(os.path.join(image_path,'record_{}_distribution.png').format(record_id), dpi=300)
    fig = draw_hit_histogram(record_hits, detector)
    fig.savefig(os.path.join(image_path, 'record_{}_histogram.png').format(record_id), dpi=300)
    fig = get_detector_scatter3ds(detector, include_modules=False, include_pmts=True, hits=record_hits, sources=record_sources)
    fig.write_image(os.path.join(image_path, 'record_{}_3d.png').format(record_id), scale=2)

    